In [ ]:
# Imports
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import psutil
import os
from sklearn.base import BaseEstimator, ClassifierMixin

In [ ]:
# Data Loading and Preprocessing ( preprocessed assumption )
# data = preprocessed data pickels/csvs here
# X = data.data
# y = data.target
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


CONVERTING TO TENSORS

In [ ]:
# Simple NN Implementation 
class SimpleNN(BaseEstimator, ClassifierMixin):
    def __init__(self, hidden_layer_sizes=(8,), activation='relu'):
        self.hidden_layer_sizes = hidden_layer_sizes
        self.activation = activation

    def fit(self, X, y):
        self.layers = []
        layer_sizes = [X.shape[1]] + list(self.hidden_layer_sizes) + [len(np.unique(y))]
        for i in range(len(layer_sizes) - 1):
            W = np.random.randn(layer_sizes[i], layer_sizes[i + 1]) * 0.1
            b = np.zeros((1, layer_sizes[i + 1]))
            self.layers.append((W, b))
        return self

    def _activate(self, x):
        if self.activation == 'relu':
            return np.maximum(0, x)
        elif self.activation == 'tanh':
            return np.tanh(x)
        else:
            raise ValueError("Unsupported activation function")

    def softmax(self, x):
        e_x = np.exp(x - np.max(x, axis=1, keepdims=True))
        return e_x / e_x.sum(axis=1, keepdims=True)

    def predict_proba(self, X):
        a = X
        for i, (W, b) in enumerate(self.layers[:-1]):
            z = np.dot(a, W) + b
            a = self._activate(z)
        z = np.dot(a, self.layers[-1][0]) + self.layers[-1][1]
        return self.softmax(z)

    def predict(self, X):
        return np.argmax(self.predict_proba(X), axis=1)

In [ ]:
# PyTorch NN
class TorchNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(TorchNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [ ]:
# LLM Inspired Robust NN (non-torch version)
class RobustNN(BaseEstimator, ClassifierMixin):
    def __init__(self, hidden_layer_sizes=(16, 16), activation='relu'):
        self.hidden_layer_sizes = hidden_layer_sizes
        self.activation = activation

    def fit(self, X, y):
        self.layers = []
        layer_sizes = [X.shape[1]] + list(self.hidden_layer_sizes) + [len(np.unique(y))]
        for i in range(len(layer_sizes) - 1):
            W = np.random.randn(layer_sizes[i], layer_sizes[i + 1]) * 0.1
            b = np.zeros((1, layer_sizes[i + 1]))
            self.layers.append((W, b))
        return self

    def _activate(self, x):
        if self.activation == 'relu':
            return np.maximum(0, x)
        elif self.activation == 'tanh':
            return np.tanh(x)
        else:
            raise ValueError("Unsupported activation function")

    def dropout(self, x, drop_rate=0.2):
        mask = np.random.binomial(1, 1 - drop_rate, size=x.shape)
        return x * mask / (1 - drop_rate)

    def softmax(self, x):
        e_x = np.exp(x - np.max(x, axis=1, keepdims=True))
        return e_x / e_x.sum(axis=1, keepdims=True)

    def predict_proba(self, X):
        a = X
        for i, (W, b) in enumerate(self.layers[:-1]):
            z = np.dot(a, W) + b
            a = self._activate(z)
            a = self.dropout(a)
        z = np.dot(a, self.layers[-1][0]) + self.layers[-1][1]
        return self.softmax(z)

    def predict(self, X):
        return np.argmax(self.predict_proba(X), axis=1)

In [ ]:
# Parameter and RAM Usage Function
def model_info_sklearn(model):
    if hasattr(model, 'coefs_') and hasattr(model, 'intercepts_'):
        total_params = sum(w.size for w in model.coefs_) + sum(b.size for b in model.intercepts_)
    elif hasattr(model, 'layers'):
        total_params = sum(W.size + b.size for W, b in model.layers)
    else:
        total_params = 0
    process = psutil.Process(os.getpid())
    ram_usage = process.memory_info().rss / (1024 ** 2)  # in MB
    return total_params, ram_usage


In [ ]:
# Shared Grid Search Configuration
shared_param_grid = {
    'hidden_layer_sizes': [
        (8,), (16,), (32,),
        (8, 8), (16, 16), (32, 32),
        (8, 16), (16, 8), (8, 32), (32, 8),
        (16, 32), (32, 16), (8, 16, 8), (16, 8, 16)
    ],
    'activation': ['relu', 'tanh']
}

# Grid Search for All Models
models = {
    'MLPClassifier': MLPClassifier(max_iter=500, random_state=42),
    'SimpleNN': SimpleNN(),
    'RobustNN': RobustNN()
}

In [ ]:
for name, model in models.items():
    gs = GridSearchCV(model, shared_param_grid, cv=3)
    gs.fit(X_train, y_train)
    pred = gs.predict(X_test)
    acc = accuracy_score(y_test, pred)
    params, ram = model_info_sklearn(gs.best_estimator_)
    print(f"{name} - Accuracy: {acc:.4f}, Total Parameters: {params}, RAM Usage: {ram:.2f} MB")